In [ ]:
import time
import requests
import json
import hmac
import hashlib


class api_v2(object):
    _api_url = 'https://api.bitfinex.com'
    _api_key = ''
    _api_secret = ''
    _api_nonce = ''

    # Init class
    def init(self, api_key=None, api_secret=None):
        if api_key is None and api_secret is None:
            print('No keys, only access to public API functions')
        else:
            self._api_key = api_key
            self._api_secret = api_secret

    # create nonce
    @staticmethod
    def _nonce():
        return str(int(round(time.time() * 1000)))

    def _headers(self, path, nonce, body):

        signature = '/api' + path + nonce + body
        print('Signing: ' + signature)

        h = hmac.new(self._api_secret.encode('utf-8'), signature.encode('utf-8'), hashlib.sha384)
        signature = h.hexdigest().lower()

        return {
            'bfx-nonce': nonce,
            'bfx-apikey': self._api_key,
            'bfx-signature': signature,
            'content-type': 'application/json'
        }

    # create signature
    def api_call(self, method, param={}):
        url = self._api_url + method
        nonce = self._nonce()
        raw_body = json.dumps(param)
        headers = self._headers(method, nonce, raw_body)

        # return requests.post(url, data={}, headers=headers)
        return requests.post(url, headers=headers, data=raw_body, verify=True)

    # Public endpoints
    @staticmethod
    def tickers(symbol='btcusd'):
        return requests.get('https://api.bitfinex.com/v2/tickers?symbols=t{}'.
                            format(symbol.upper())).json()

    @staticmethod
    def ticker(symbol='btcusd'):
        return requests.get('https://api.bitfinex.com/v2/ticker/t{}'.
                            format(symbol.upper())).json()

    @staticmethod
    def trades(symbols='btcusd', limit=1000, start=None, end=None, sort=-1):
        return requests.get('https://api.bitfinex.com/v2/trades/t{}/hist?limit={}&start={}&end={}&sort=-1'.
                            format(symbols.upper(), limit, start, end, sort)).json()

    @staticmethod
    def books(symbol='btcusd', precision='P0', length=100):
        return requests.get('https://api.bitfinex.com/v2/book/t{}/{}?len={}'.
                            format(symbol.upper(), precision, length)).json()

    @staticmethod
    def stats(key='funding.size', size='1m', symbol='usd', sort=-1):
        return requests.get('https://api.bitfinex.com/v2/stats1/{}:{}:f{}/hist?sort={}'.
                            format(key, size, symbol.upper(), sort)).json()

    @staticmethod
    def candles(symbol='btcusd', interval='1m', limit=1000, start=None, end=None, sort=-1):
        return requests.get('https://api.bitfinex.com/v2/candles/trade:{}:t{}/hist?limit={}&start={}&end={}&sort=-1'.
                            format(interval, symbol.upper(), limit, start, end, sort)).json()

    # REST calculation endpoints
    @staticmethod
    def market_average_price(symbol='btcusd', amount=1.123, period='', rate_limit=''):
        url = 'https://api.bitfinex.com/v2/calc/trade/avg'
        querystring = {'symbol': 't'+str(symbol).upper(),
                       'amount': str(amount),
                       'period': str(period),
                       'rate_limit': str(rate_limit)}
        return requests.request('POST', url, params=querystring).json()

    @staticmethod
    def forex(ccy1='eur', ccy2='usd'):
        url = 'https://api.bitfinex.com/v2/calc/fx'
        querystring = {'ccy1': ccy1.upper(),
                       'ccy2': ccy2.upper()}
        return requests.request('POST', url, params=querystring).json()

    # Private endpoints
    def wallets(self):
        return self.api_call('/v2/auth/r/wallets', {}).json()

    def orders(self, symbol='btcusd', start=1545079680000, end=1545019080000, limit=25, sort=-1):
        querystring = {'symbol': 't'+str(symbol).upper(),
                       'start': str(start),
                       'end': str(end),
                       'limit': str(limit),
                       'sort': str(sort)}
        return self.api_call('/v2/auth/r/orders/t{}/hist'.format(symbol.upper()), querystring).json()

    def order_trades(self, symbol='btcusd', order_id=''):
        return self.api_call('/v2/auth/r/order/t{}:{}/trades'.format(symbol.upper(), order_id), {}).json()

    def positions(self):
        return self.api_call('/v2/auth/r/positions', {}).json()

    def funding_offers(self, symbol='btcusd'):
        return self.api_call('/v2/auth/r/funding/offers/t{}'.format(symbol.upper()), {}).json()

    def funding_offers_history(self, symbol='btcusd', start=1545079680000, end=1545019080000, limit=25):
        querystring = {'symbol': 't'+str(symbol).upper(),
                       'start': str(start),
                       'end': str(end),
                       'limit': str(limit)}
        return self.api_call('/v2/auth/r/funding/offers/t{}/hist'.format(symbol.upper()), querystring).json()

    def loans(self, symbol='btcusd'):
        return self.api_call('/v2/auth/r/funding/loans/t{}'.format(symbol.upper()), {}).json()

    def loans_history(self, symbol='btcusd', start=1545079680000, end=1545019080000, limit=25):
        querystring = {'symbol': 't'+str(symbol).upper(),
                       'start': str(start),
                       'end': str(end),
                       'limit': str(limit)}
        return self.api_call('/v2/auth/r/funding/loans/t{}/hist'.format(symbol.upper()), querystring).json()

    def credits(self, symbol='btcusd'):
        return self.api_call('/v2/auth/r/funding/credits/t{}'.format(symbol.upper()), {}).json()

    def credits_history(self, symbol='btcusd', start=1545079680000, end=1545019080000, limit=25):
        querystring = {'symbol': 't'+str(symbol).upper(),
                       'start': str(start),
                       'end': str(end),
                       'limit': str(limit)}
        return self.api_call('/v2/auth/r/funding/credits/t{}/hist'.format(symbol.upper()), querystring).json()

    def funding_trades(self, symbol='btcusd', start=1545079680000, end=1545019080000, limit=25):
        querystring = {'symbol': 't'+str(symbol).upper(),
                       'start': str(start),
                       'end': str(end),
                       'limit': str(limit)}
        return self.api_call('/v2/auth/r/funding/trades/t{}/hist'.format(symbol.upper()), querystring).json()

    def margin_info(self, key=''):
        return self.api_call('/v2/auth/r/info/margin/{}'.format(key), {}).json()

    def funding_info(self, key=''):
        return self.api_call('/v2/auth/r/info/funding/{}'.format(key), {}).json()

    def movements(self, currency='btc'):
        return self.api_call('/v2/auth/r/movements/{}/hist'.format(currency.upper()), {}).json()

    def alerts(self, alert_type='price'):
        querystring = {'type': alert_type}
        return self.api_call('/v2/auth/r/alerts', querystring).json()

    def alert_set(self, alert_type='price', symbol='btc', price=1000.0):
        querystring = {'type': alert_type,
                       'symbol': symbol.upper(),
                       'price': str(price)}
        return self.api_call('/v2/auth/w/alert/set', querystring).json()

    def alert_delete(self, symbol='btc', price=1000.0):
        return self.api_call('/v2/auth/w/alert/price:t{}:{}/del'.format(symbol.upper(), price), {}).json()

    def calc_available_balance(self, symbol='btcusd', direction=1, rate=1, order_type='EXCHANGE'):
        querystring = {'symbol': 't'+str(symbol).upper(),
                       'dir': str(direction),
                       'rate': str(rate),
                       'type': order_type.upper()}
        return self.api_call('/v2/auth/calc/order/avail/', querystring).json()

    def ledgers(self, currency='btc'):
        return self.api_call('/v2/auth/r/ledgers/{}/hist'.format(currency.upper()), {}).json()

    def user_settings_read(self, params={}):
        return self.api_call('/v2/auth/r/settings', params).json()

    def user_settings_write(self, params={}):
        return self.api_call('/v2/auth/w/settings/set', params).json()

    def user_settings_delete(self, params={}):
        return self.api_call('/v2/auth/w/settings/del', params).json()

    def user_info(self):
        return self.api_call('/v2/auth/r/info/user', {}).json()

In [ ]:
import datetime
import time
#!pip install bitfinex
import bitfinex

class Bitfinex(object):
    def init(self):
        self.limit = 10000

    def request_candles(self, market, start_time, end_time, candle_length, candle_prior):
        start_time = datetime.datetime(start_time[0], start_time[1], start_time[2], start_time[3], start_time[4])
        end_time = datetime.datetime(end_time[0], end_time[1], end_time[2], end_time[3], end_time[4])

        if candle_length == '1D':
            t_start_before = start_time - datetime.timedelta(days=candle_prior)
        elif candle_length == '7D':
            t_start_before = start_time - datetime.timedelta(days=candle_prior * 7)
        elif candle_length == '14D':
            t_start_before = start_time - datetime.timedelta(days=candle_prior * 14)
        elif candle_length == '1M':
            t_start_before = start_time - datetime.timedelta(weeks=candle_prior * 4)
        elif candle_length == '1m':
            t_start_before = start_time - datetime.timedelta(minutes=candle_prior)
        elif candle_length == '5m':
            t_start_before = start_time - datetime.timedelta(minutes=candle_prior * 5)
        elif candle_length == '15m':
            t_start_before = start_time - datetime.timedelta(minutes=candle_prior * 15)
        elif candle_length == '30m':
            t_start_before = start_time - datetime.timedelta(minutes=candle_prior * 30)
        elif candle_length == '1h':
            t_start_before = start_time - datetime.timedelta(hours=candle_prior)
        elif candle_length == '3h':
            t_start_before = start_time - datetime.timedelta(hours=candle_prior * 3)
        elif candle_length == '6h':
            t_start_before = start_time - datetime.timedelta(hours=candle_prior * 6)
        elif candle_length == '12h':
            t_start_before = start_time - datetime.timedelta(hours=candle_prior * 12)

        apiV2 = api_v2()
        t_start_before = time.mktime(t_start_before.timetuple()) * 1000

        t_stop = time.mktime(end_time.timetuple()) * 1000

        candles = apiV2.candles(symbol=market, interval=candle_length, limit=10000
                                 , start=t_start_before, end=t_stop)

        return candles


In [ ]:
# Candles which start from hours other than 00:00
def time_maker(timestamp):
  return [datetime.datetime.utcfromtimestamp(timestamp).year,
          datetime.datetime.utcfromtimestamp(timestamp).month,
          datetime.datetime.utcfromtimestamp(timestamp).day,
          datetime.datetime.utcfromtimestamp(timestamp).hour,
          0]

#   bitfinex = Bitfinex()
#   candle = bitfinex.request_candles(symbol, time, time_plus, '1h', 0)

#   prices = []
#   print(candle)
#   for i in range(len(candle)):
#     for j in range(1, 5):
#       prices.append(round(candle[i][j], 1))

#   return [candle[0][1], candle[len(candle)-1][4],
#           max(prices), min(prices)] # Open, Close, High, Low

# def candle_generator(timestamp, symbol):
#   time = [datetime.datetime.utcfromtimestamp(timestamp).year,
#           datetime.datetime.utcfromtimestamp(timestamp).month,
#           datetime.datetime.utcfromtimestamp(timestamp).day,
#           datetime.datetime.utcfromtimestamp(timestamp).hour,
#           0]
#   time_plus = [datetime.datetime.utcfromtimestamp(timestamp + 86400).year,
#           datetime.datetime.utcfromtimestamp(timestamp + 86400).month,
#           datetime.datetime.utcfromtimestamp(timestamp + 86400).day,
#           datetime.datetime.utcfromtimestamp(timestamp + 86400).hour,
#           0]

#   bitfinex = Bitfinex()
#   candle = bitfinex.request_candles(symbol, time, time_plus, '1D', 0)
  
#   return candle[len(candle)-1] # Open, Close, High, Low
  
#print(candle_generator(1619568011, 'ltcusd'))


In [ ]:
import pandas as pd
import numpy as np

btc_data = pd.read_csv('btc_data.csv')
btc_data.drop('Unnamed: 0', inplace=True, axis=1)

# btc_data = pd.DataFrame(columns=['source', 'published_on', 'title', 'body', 'tags'])

# for i in range(len(nonlabeled_data)):
#   if nonlabeled_data['tags'][i] == 'btcusd':
#     btc_data = btc_data.append(nonlabeled_data.iloc[i], ignore_index=True)

for i in range(len(btc_data)):
  if btc_data['published_on'][i] % 3600 < 1800:
    btc_data['published_on'][i] -= (btc_data['published_on'][i] % 3600)
  else:
    btc_data['published_on'][i] -= ((btc_data['published_on'][i] % 3600) - 3600 )

since = time_maker(min(btc_data['published_on']))
to = time_maker(max(btc_data['published_on']))

B = Bitfinex()

btc_candles = B.request_candles('btcusd', since, to, '1h', 0)

price_dict = dict()

for i in range(len(btc_candles)):
  price_dict[btc_candles[i][0]] = [round(btc_candles[i][1]),
                                   round(btc_candles[i][2]),
                                   round(btc_candles[i][3]),
                                   round(btc_candles[i][4])]

btc_data['return'] = 0.0

for i in range(39, 13594):
    news_return = round(100*(price_dict[(btc_data['published_on'][i]*1000+86400000)][2] - price_dict[(btc_data['published_on'][i]*1000)][1])/price_dict[btc_data['published_on'][i]*1000][1], 3)
    btc_data['return'][i] = news_return

for i in range(len(btc_data)):
  if btc_data['return'][i] == 0:
    btc_data = btc_data.drop([i])

# What about removing s from end of each word?!

btc_data = btc_data.sample(frac=1)[0:13500]
btc_data = btc_data.reset_index()

for i in range(len(btc_data)):
  try:
    btc_data['body'][i] = btc_data['title'][i] + ' ' + btc_data['body'][i]
  except:
    btc_data = btc_data.drop([i])

btc_data = btc_data.drop(columns=['index', 'source', 'published_on', 'tags', 'title'])
btc_data = btc_data.rename(columns={'body':"article"})
btc_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: Settin

,article,return
0,bitcoin price experiencing strong market corre...,2.224
1,surge bitcoin energy consumption sparks debate...,6.900
2,bitcoin expects fall another asserts veteran t...,1.100
3,nasdaqlisted bit digital acquires bitcoin mine...,1.868
4,decadeold idle bitcoins worth million moved re...,3.672
...,...,...
13495,bitcoin likely remain ultra volatile due limit...,-6.793
13496,could probably hider bitcoins bull rally bitco...,5.557
13497,nearly bitcoin supply hasnt budged years bitco...,8.280
13498,driving wrapped bitcoins growth year turned qu...,-1.642


In [ ]:
import numpy as np

# selected_data['return'] = 0
# for i in range(len(selected_data)):
#   print(i)
#   corresponding_candle = candle_generator(selected_data['published_on'][i], selected_data['tags'][i])
#   label = 100*(corresponding_candle[1]-corresponding_candle[0])/corresponding_candle[0]
#   selected_data['return'][i] = label
#   time.sleep(0.2)

D = np.array([[1, 2, 3],
              [0, 2, 1],
              [1, 1, 1],
              [0, 0, 5]])

print(len(D))

4


In [ ]:
from google.colab import drive

btc_data.to_csv('final_data.csv')
!cp final_data.csv "drive/My Drive/"